In [3]:
cd Downloads/nptel-pure

/Users/riyasachdeva/Downloads/nptel-pure


In [4]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

zsh:1: no matches found: datasets[audio]


In [5]:
!pip install openai-whisper

In [6]:
!pip install datasets

In [7]:
!pip install jiwer

In [8]:
import whisper
from datasets import DatasetDict, load_dataset
from jiwer import cer
import numpy as np

# Initialize the Whisper model
model = whisper.load_model("base")

In [9]:
def transcribe_and_calculate_cer(model, dataset):
    transcriptions = []
    cers = []
    i = 0
    for example in dataset['validation']:
        # Get the audio file and the ground truth transcription
        audio = example['audio']['array']
        ground_truth = example['text']

        # Ensure audio data is in float32 format
        audio = np.array(audio, dtype=np.float32)  # Convert to float32

        # Transcribe the audio file
        print(f"Transcribing audio...{i}")
        result = model.transcribe(audio)
        transcription = result["text"]

        # Calculate CER
        error_rate = cer(ground_truth.lower(), transcription.lower())

        # Append results
        transcriptions.append(transcription)
        cers.append(error_rate)
        i += 1

    return dataset, transcriptions, cers

In [10]:
from datasets import Dataset, DatasetDict, Audio
import os

# Directories containing the wav files and corresponding text files
audio_dir = "./wav"
text_dir = "./corrected_txt"

# Function to load audio and text files
def load_data(audio_dir, text_dir):
    data = {"audio": [], "text": []}
    for audio_file in os.listdir(audio_dir):
        if audio_file.endswith(".wav"):
            # Extract the base name (without extension)
            base_name = os.path.splitext(audio_file)[0]
            text_file = base_name + ".txt"

            # Check if corresponding text file exists and is not empty
            text_file_path = os.path.join(text_dir, text_file)
            if os.path.isfile(text_file_path):
                with open(text_file_path, "r", encoding="utf-8") as f:
                    text_content = f.read().strip()
                    if text_content:  # Ensure the text content is not empty
                        data["audio"].append(os.path.join(audio_dir, audio_file))
                        data["text"].append(text_content)

    return data

# Load data
data = load_data(audio_dir, text_dir)

# Create the Dataset
dataset = Dataset.from_dict(data)
dataset = dataset.cast_column("audio", Audio())

# Create the DatasetDict
dataset_dict = DatasetDict({"validation": dataset})

# Print the DatasetDict
print(dataset_dict)


DatasetDict({
    validation: Dataset({
        features: ['audio', 'text'],
        num_rows: 998
    })
})


In [11]:
!pip install librosa

In [12]:
result_dataset, transcriptions, cers = transcribe_and_calculate_cer(model, dataset_dict)

Transcribing audio...0


/Users/riyasachdeva/anaconda3/envs/plankton/lib/python3.9/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribing audio...1
Transcribing audio...2
Transcribing audio...3
Transcribing audio...4
Transcribing audio...5
Transcribing audio...6
Transcribing audio...7
Transcribing audio...8
Transcribing audio...9
Transcribing audio...10
Transcribing audio...11
Transcribing audio...12
Transcribing audio...13
Transcribing audio...14
Transcribing audio...15
Transcribing audio...16
Transcribing audio...17
Transcribing audio...18
Transcribing audio...19
Transcribing audio...20
Transcribing audio...21
Transcribing audio...22
Transcribing audio...23
Transcribing audio...24
Transcribing audio...25
Transcribing audio...26
Transcribing audio...27
Transcribing audio...28
Transcribing audio...29
Transcribing audio...30
Transcribing audio...31
Transcribing audio...32
Transcribing audio...33
Transcribing audio...34
Transcribing audio...35
Transcribing audio...36
Transcribing audio...37
Transcribing audio...38
Transcribing audio...39
Transcribing audio...40
Transcribing audio...41
Transcribing audio...42
T

In [13]:
result_dataset['validation'][0]

{'audio': {'path': './wav/00082854c239062d386c13bf5bca0bf9f8322381080e0c3ef99f460c.wav',
  'array': array([ 0.03305054,  0.0272522 ,  0.02218628, ..., -0.04318237,
         -0.021698  ,  0.01885986]),
  'sampling_rate': 16000},
 'text': 'EVENT HAS NOT CHANGED BECAUSE BOTH THESE EVENTS OCCURRED EXACTLY AT THE SAME POINT THE LIGHT IS EMITTED FROM THIS POINT GOES UPS GOES UP AND COMES'}

In [14]:
cer = 0
for i in range(998):
    print(f"Audio {i+1}:")
    print(f"Ground Truth: {result_dataset['validation'][i]['text']}")
    print(f"Transcription: {transcriptions[i]}")
    print(f"CER: {cers[i]}\n")
    cer += cers[i]
print(f"Average CER: {cer/998}")

Audio 1:
Ground Truth: EVENT HAS NOT CHANGED BECAUSE BOTH THESE EVENTS OCCURRED EXACTLY AT THE SAME POINT THE LIGHT IS EMITTED FROM THIS POINT GOES UPS GOES UP AND COMES
Transcription:  event has not changed because both these events occur exactly at the same point. The light is emitted from this point, goes up, goes up and comes out.
CER: 0.07534246575342465

Audio 2:
Ground Truth: OR A COLLECTION OF MULTIPLE ELEMENTS WHICH ARE KNOWN AS BLOBS SO ONE IMPORTANT TERM
Transcription:  or a collection of multiple elements which are known as blobs. So, one important term is,
CER: 0.07228915662650602

Audio 3:
Ground Truth: WE WANT OK THIS IS IMPORTANT THIS IS IMPORTANT SO IN CASE WHEN MU IS A REAL EIGEN VALUE
Transcription:  to 1 ok this is important this is important. In case when mu is a real eigenvalue, it is a real eigenvalue.
CER: 0.39080459770114945

Audio 4:
Ground Truth: SO IT IS A FINITE SERIES THAT IS THE ESSENTIAL PART OF THE DEFINITION IT IS A FINITE SERIES CONTAINING
Transcripti

In [ ]:

min_cer = min(cers)
print(f"BEST CER: {min_cer}")
print(f"Average CER: {cer/73}")
